In [1]:
from glob import glob
from os.path import join
import pandas as pd

from modules.imgur import Client
from modules.metadata import PhotoMetadata

metadata_dirpath = '../../data'
photo_metadata_dirpath = '../../data/photo_metadata'
photo_metadata_path = join(photo_metadata_dirpath, 'photo_metadata.hdf')
imgur_data_path = join(photo_metadata_dirpath, 'imgur_data.hdf')

%reload_ext autoreload
%autoreload 2

In [ ]:
# RENAME IMGUR ALBUM
#OLD_TITLE = 'Morocco (2019)'
#NEW_TITLE = 'Morocco'
#ALBUM_HASH = client.album_dict[OLD_TITLE]
#response = client.update_album(ALBUM_HASH, {'ids': None, 'title': NEW_TITLE})

In [2]:
import json
with open(join(metadata_dirpath, 'imgur_api.json'), 'r') as file:
    api_keys = json.load(file)
    
CLIENT_ID = api_keys['CLIENT_ID']
CLIENT_SECRET = api_keys['CLIENT_SECRET']
ACCESS_TOKEN = api_keys['ACCESS_TOKEN']
REFRESH_TOKEN = api_keys['REFRESH_TOKEN']

client = Client(imgur_data_path, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, REFRESH_TOKEN)

In [37]:
# # generate new access token
# client = Client(imgur_data_path, CLIENT_ID, CLIENT_SECRET)
# auth_url = client.get_auth_url('token')
# token = client.get_auth_url('token')
# client.set_user_auth(params['access_token'], params['refresh_token'])

# Upload photos

In [16]:
START_DATE = '2020-04-06'
photo_metadata = PhotoMetadata(photo_metadata_path)
records_to_add = photo_metadata.data[photo_metadata.render_times > START_DATE]

In [18]:
from modules.utilities import str_to_datetime
import numpy as np

def is_later(record):
    if type(record.time_uploaded) == float and np.isnan(record.time_uploaded):
        return True
    else:
        return str_to_datetime(record.time_uploaded) < str_to_datetime(record.time_rendered)
    
records_to_add = records_to_add[records_to_add.join(client.imgur_data).apply(is_later, axis=1)]

In [ ]:
client.upload_new_images(photo_metadata)